## Battle of Neighborhoods in New York City

##### Table of Contents


1. Step 1: Import Libraries</a>
2. Step 2: Import and Arrange Data</a>
3. Step 3: Geographical Exploration of New York City</a>
4. Step 4: Geographical Exploration of Three New York Boroughs</a>
5. Step 5: Using Foursquare API to Explore the Boroughs</a> 
6. Step 6: Explore Neighborhoods in Manhattan, Brookyln, and Queens</a> 
7. Step 7: Final Report</a> 

#### Step 1: import Libraries

In [388]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Step 2: Import and Arrange Data 

In [300]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [301]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [302]:
#Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.
neighborhoods_data = newyork_data['features']

In [303]:
#Let's take a look at the first item in this list.
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Tranform the data into a pandas dataframe. This is essentially transforming this data of nested Python dictionaries into a pandas dataframe. Start by creating an empty dataframe.

In [304]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [305]:
#Take a look at the empty dataframe to confirm that the columns are as intended.
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [306]:
#Then let's loop through the data and fill the dataframe one row at a time.

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [307]:
print ('shape of data:', neighborhoods.shape)
neighborhoods.head()

shape of data: (306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [308]:
#Make sure that the dataset has all 5 boroughs and 306 neighborhoods.

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


#### Step 3: Geographical Exploration of New York City

Use geopy library to get the latitude and longitude values of New York City.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [309]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [310]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Step 4: Geographical Exploration of Three New York Boroughs

Slice the original dataframe and create new dataframes for Manhattan, Brooklyn, and Queens. Manhattan is the biggest commercial borough. Queens has the largest land mass followed by Brooklyn. In addition, the target audience for opening a new African resturant is the Queens and Brooklyn.

In [311]:
#Manhattan data
Manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
print('shape of Manhattan_data is:', Manhattan_data.shape)
Manhattan_data.head()

shape of Manhattan_data is: (40, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [312]:
#Brooklyn Data
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
print('shape of Brooklyn_data is:', Brooklyn_data.shape)
Brooklyn_data.head()

shape of Brooklyn_data is: (70, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [313]:
#Queens Data
Queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
print('shape of Brooklyn_data is:', Queens_data.shape)
Queens_data.head()

shape of Brooklyn_data is: (81, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


Next, we get the geographical coordinates of the three boroughs

In [314]:
#Let's get the geographical coordinates of Manhattan
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [315]:
#Let's get the geographical coordinates of Brooklyn
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [316]:
#Let's get the geographical coordinates of Manhattan
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


After getting the geographical coordinates of these three Borough, we creat maps for these three boroughs for proper visualization.

In [317]:
# create map of Manhattan using latitude and longitude values
Manhattan_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Manhattan_data['Latitude'], Manhattan_data['Longitude'], Manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Manhattan_map)  
    
Manhattan_map

In [318]:
# create map of Brooklyn using latitude and longitude values
Brooklyn_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Brooklyn_map)  
    
Brooklyn_map

In [319]:
# create map of Queens using latitude and longitude values
Queens_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Queens_data['Latitude'], Queens_data['Longitude'], Queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Queens_map)  
    
Queens_map

#### Step 5: Using Foursquare API to Explore the Boroughs

The Foursquare API will be used to explore the two Boroughs. First, establish Foursquare credentials.

In [320]:
CLIENT_ID = 'OVRXU0L3BTQWDGLVIFS1LQZONX0YUIDMBR1QUKZIYVILNAII'
CLIENT_SECRET = '0TD3IABDCRIQ0AANXYQJB0AYFWSKPHP15HR41O2LMLOUXLY4' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OVRXU0L3BTQWDGLVIFS1LQZONX0YUIDMBR1QUKZIYVILNAII
CLIENT_SECRET:0TD3IABDCRIQ0AANXYQJB0AYFWSKPHP15HR41O2LMLOUXLY4


Next, explore the first neighborhoods in the Manhattan, Brookkyn, and Queens dataframes.

In [322]:
Manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [323]:
Brooklyn_data.loc[0, 'Neighborhood']

'Bay Ridge'

In [325]:
Queens_data.loc[0, 'Neighborhood']

'Astoria'

Next we obtain the geographical coordinates of these first neighborhoods.

In [328]:
#Get the Marble Hill latitude and longitude values
Marblehill_latitude = Manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
Marblehill_longitude = Manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               Marblehill_latitude, 
                                                               Marblehill_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [236]:
#Get the Bay Ridge latitude and longitude values
BayRidge_latitude = Brooklyn_data.loc[0, 'Latitude'] # neighborhood latitude value
BayRidge_longitude = Brooklyn_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Brooklyn_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               BayRidge_latitude, 
                                                               BayRidge_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


In [329]:
#Get the Astoria latitude and longitude values
Astoria_latitude = Queens_data.loc[0, 'Latitude'] # neighborhood latitude value
Astoria_longitude = Queens_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Queens_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               Astoria_latitude, 
                                                               Astoria_longitude))

Latitude and longitude values of Astoria are 40.76850859335492, -73.91565374304234.


Next we get the top 100 venues that are in Marble Hill, Bay Ridge, and Astoria within a radius of 1610 meters (1 mile).

In [330]:
#create the GET request URL for Marble Hill. Name the URL url.
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1610 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Marblehill_latitude, 
    Marblehill_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OVRXU0L3BTQWDGLVIFS1LQZONX0YUIDMBR1QUKZIYVILNAII&client_secret=0TD3IABDCRIQ0AANXYQJB0AYFWSKPHP15HR41O2LMLOUXLY4&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=1610&limit=100'

In [331]:
#Send the GET request and examine the Manhattan resutls
results_Manhattan = requests.get(url).json()
results_Manhattan

{'meta': {'code': 200, 'requestId': '5ecdebe971c428001b10b482'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 131,
  'suggestedBounds': {'ne': {'lat': 40.891040793289655,
    'lng': -73.89153182999516},
   'sw': {'lat': 40.862060764309625, 'lng': -73.92978748726446}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4baf59e8f964a520a6f93be3',
       'name': 'Bikram Yoga',
       'location': {'address': '5500 Broadway',
        'crossStreet': '230th Street',
        'lat': 40.876843690797934,
        'lng': -73.9062

In [332]:
#create the GET request URL for Bay Ridge. Name the URL url.
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1610 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    BayRidge_latitude, 
    BayRidge_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OVRXU0L3BTQWDGLVIFS1LQZONX0YUIDMBR1QUKZIYVILNAII&client_secret=0TD3IABDCRIQ0AANXYQJB0AYFWSKPHP15HR41O2LMLOUXLY4&v=20180605&ll=40.625801065010656,-74.03062069353813&radius=1610&limit=100'

In [333]:
#Send the GET request and examine the Brooklyn resutls
results_Brooklyn = requests.get(url).json()
results_Brooklyn

{'meta': {'code': 200, 'requestId': '5ecded8e77af03001b313c3a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Bay Ridge',
  'headerFullLocation': 'Bay Ridge, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 189,
  'suggestedBounds': {'ne': {'lat': 40.64029107950067,
    'lng': -74.0115648624278},
   'sw': {'lat': 40.61131105052064, 'lng': -74.04967652464845}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad09cf7f964a520bed820e3',
       'name': 'Bagel Boy',
       'location': {'address': '8002 3rd Ave',
        'crossStreet': '80th St',
        'lat': 40.627896,
        'lng': -74.029335,
        'labeledLatL

In [334]:
#create the GET request URL for Astoria. Name the URL url.
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1610 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Astoria_latitude, 
    Astoria_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OVRXU0L3BTQWDGLVIFS1LQZONX0YUIDMBR1QUKZIYVILNAII&client_secret=0TD3IABDCRIQ0AANXYQJB0AYFWSKPHP15HR41O2LMLOUXLY4&v=20180605&ll=40.76850859335492,-73.91565374304234&radius=1610&limit=100'

In [335]:
#Send the GET request and examine the Brooklyn resutls
results_Queens = requests.get(url).json()
results_Queens

{'meta': {'code': 200, 'requestId': '5ecded76216785001b0bb4ef'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Astoria',
  'headerFullLocation': 'Astoria, Queens',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 239,
  'suggestedBounds': {'ne': {'lat': 40.78299860784494,
    'lng': -73.89655704790823},
   'sw': {'lat': 40.75401857886491, 'lng': -73.93475043817645}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a9c0105f964a520b03520e3',
       'name': 'Titan Foods Inc.',
       'location': {'address': '25-56 31st St',
        'crossStreet': 'btwn 28th Rd. & Newtown Ave.',
        'lat': 40.76919777900986,
        'lng': 

All the information required is in the items key. We can use the get_category_type function, clean the data and structure it into a pandas dataframe.

In [336]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json's and structure them into a pandas dataframe for Manhattan, Brooklyn, and Queens.

In [339]:
#Venues for Manhattan
venues = results_Manhattan['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print('Shape of near by venues in Manhattan:', nearby_venues.shape)
print('{} venues in Manhattan were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

Shape of near by venues in Manhattan: (100, 4)
100 venues in Manhattan were returned by Foursquare.


,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Sam's Pizza,Pizza Place,40.879435,-73.905859
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Arturo's,Pizza Place,40.874412,-73.910271
4,El Malecon,Caribbean Restaurant,40.879338,-73.904457


In [340]:
#Venues for Brooklyn
venues = results_Brooklyn['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print('Shape of near by venues in Brooklyn:', nearby_venues.shape)
print('{} venues in Brookyln were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

Shape of near by venues in Brooklyn: (100, 4)
100 venues in Brookyln were returned by Foursquare.


,name,categories,lat,lng
0,Bagel Boy,Bagel Shop,40.627896,-74.029335
1,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
2,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186
3,Ho' Brah Taco Joint,Taco Place,40.622960,-74.031371
4,Karam,Middle Eastern Restaurant,40.622931,-74.028316


In [341]:
#Venues for Queens
venues = results_Queens['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print('Shape of near by venues in Queens:', nearby_venues.shape)
print('{} venues in Queens were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

Shape of near by venues in Queens: (100, 4)
100 venues in Queens were returned by Foursquare.


,name,categories,lat,lng
0,Titan Foods Inc.,Gourmet Shop,40.769198,-73.919253
1,CrossFit Queens,Gym,40.769404,-73.918977
2,Favela Grill,Brazilian Restaurant,40.767348,-73.917897
3,Al-sham Sweets and Pastries,Middle Eastern Restaurant,40.768077,-73.911561
4,Sitan Muay Thai,Martial Arts Dojo,40.766108,-73.913224


#### Step 6. Explore Neighborhoods in Manhattan, Brookyln, and Queens

Create a function to repeat the same process in step 5 to all the neighborhoods in Manhattan, Brooklyn, and Queens

In [342]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Next we write the code to run the above function on each neighborhood and create new dataframes called Manhattan_venues, Brooklyn_venues, and Queens_venues.

In [347]:
Manhattan_venues = getNearbyVenues(names=Manhattan_data['Neighborhood'],
                                   latitudes=Manhattan_data['Latitude'],
                                   longitudes=Manhattan_data['Longitude'])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [348]:
Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude'])

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [349]:
Queens_venues = getNearbyVenues(names=Queens_data['Neighborhood'],
                                   latitudes=Queens_data['Latitude'],
                                   longitudes=Queens_data['Longitude'])

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


Next, We evaluate all the venues in each of the neigborhoods within each Borough starting with Manhattan

In [366]:
#Evaluate the venues in Manhattan
print(Manhattan_venues.shape)
print('This implies that there are {} unique venues.'.format(len(Manhattan_venues['Venue'].unique())))
print('There are also {} unique categories.'.format(len(Manhattan_venues['Venue Category'].unique())))
Manhattan_venues.head()

(3080, 7)
This implies that there are 2573 unique venues.
There are also 328 unique categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [371]:
#Lets view the venues by category to get a better understanding of African Resturant placement.
print('There are only 3 African Resturants')
Manhattan_venues.groupby('Venue Category').count()

There are only 3 African Resturants


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,1,1,1,1,1,1
Adult Boutique,1,1,1,1,1,1
Afghan Restaurant,1,1,1,1,1,1
African Restaurant,3,3,3,3,3,3
American Restaurant,70,70,70,70,70,70
Antique Shop,1,1,1,1,1,1
Arcade,1,1,1,1,1,1
Arepa Restaurant,3,3,3,3,3,3
Argentinian Restaurant,5,5,5,5,5,5


In [372]:
#Since, the project is focused on opeining an African Resturant, lets see the neigbohoods in Manhattan where the 3 African resturants are located
print('All 3 African Resturants are in the Central Harlem Neighborhood')
Manhattan_venues[Manhattan_venues['Venue Category']== 'African Restaurant']

All 3 African Resturants are in the Central Harlem Neighborhood


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
381,Central Harlem,40.815976,-73.943211,Ponty Bistro Harlem,40.817886,-73.941522,African Restaurant
394,Central Harlem,40.815976,-73.943211,Keur Sokhna,40.813556,-73.945001,African Restaurant
412,Central Harlem,40.815976,-73.943211,Africa Kine Restaurant,40.813728,-73.944426,African Restaurant


Evaluate the venues in Brooklyln

In [373]:
#Evaluate the venues in Brooklyn
print(Brooklyn_venues.shape)
print('This implies that there are {} unique venues.'.format(len(Brooklyn_venues['Venue'].unique())))
print('There are also {} unique categories.'.format(len(Brooklyn_venues['Venue Category'].unique())))
Brooklyn_venues.head()

(2726, 7)
This implies that there are 2214 unique venues.
There are also 282 unique categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,The Bookmark Shoppe,40.624577,-74.030562,Bookstore


In [377]:
#Lets view the venues by category to get a better understanding of African Resturant placement.
print('There is only 1 African Resturant')
Brooklyn_venues.groupby('Venue Category').count()

There is only 1 African Resturant


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,1,1,1,1,1,1
African Restaurant,1,1,1,1,1,1
Airport Terminal,1,1,1,1,1,1
American Restaurant,43,43,43,43,43,43
Animal Shelter,1,1,1,1,1,1
Antique Shop,8,8,8,8,8,8
Arepa Restaurant,2,2,2,2,2,2
Argentinian Restaurant,3,3,3,3,3,3
Art Gallery,16,16,16,16,16,16


In [378]:
#Since, the project is focused on opeining an African Resturant, lets see the neigbohood where the African resturant is located
print('The African Resturant is located in Ocean Hill Neighborhood')
Brooklyn_venues[Brooklyn_venues['Venue Category']== 'African Restaurant']

The African Resturant is located in Ocean Hill Neighborhood


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1806,Ocean Hill,40.678403,-73.913068,The Green Place,40.677016,-73.911089,African Restaurant


Evaluate the venues in Queens

In [379]:
#Evaluate the venues in Queens
print(Queens_venues.shape)
print('This implies that there are {} unique venues.'.format(len(Queens_venues['Venue'].unique())))
print('There are also {} unique categories.'.format(len(Queens_venues['Venue Category'].unique())))
Queens_venues.head()

(2070, 7)
This implies that there are 1700 unique venues.
There are also 270 unique categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [383]:
#Lets view the venues by category to get a better understanding of African Resturant placement.
Queens_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,2,2,2,2,2,2
Afghan Restaurant,2,2,2,2,2,2
American Restaurant,18,18,18,18,18,18
Antique Shop,1,1,1,1,1,1
Arepa Restaurant,4,4,4,4,4,4
Argentinian Restaurant,2,2,2,2,2,2
Art Gallery,2,2,2,2,2,2
Art Museum,1,1,1,1,1,1
Arts & Crafts Store,2,2,2,2,2,2


In [385]:
#Since, the project is focused on opeining an African Resturant, lets see the neigbohood where the African resturant is located
print('There is no African Resturant located in the Queens_venues. Therefore, this will be a strategic Borough to establish one')
Queens_venues[Queens_venues['Venue Category']== 'African Restaurant']

There is no African Resturant located in the Queens_venues. Therefore, this will be a strategic Borough to establish one


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


## Final Report:African Restaurants and Cuisines in New York City

#### Problem Description and Background

As of 2019, the population demographics show that approximately 8.3 million people reside in New York City. Among this population, 25.1% are African Americans or migrated from Africa. There are so many buisness in NYC and one that is of particular interest is to me the hospitality industry. 

One of the major reasons the hospitality industry trives in New York city is because of the population and diversity of residents. In addition, NYC is a center of attraction where people come from all over the world for tourism and buisness. There are serval buisnesses within the hospitality industry. One profitable and distinct buisness is food and catering. Given the nature of life in NYC, the average New Yorker is always on the move and more often than not, relies on eating out at restaurtants, cafe's or placing orders.

The Primary goal in this project is the evaluate the boroughs in NYC and determine which would favor the opening of an African Restaurent. The choice of an African restaurant will be discussed in the discussion section of the report. This project will be situable for culinary investors and people looking to start African catering buisness where there is little to no competition.

#### Data Description

The data that will be used in solving this problem is the New York data json file that was used in one of the assignments. The data will be be imported and arranged properly. It will be read into pandas data frame and the geographical discription of the data will be conducted. In addition, I would use my Foursquare API credentials to explore the neighborhoods within the selected boroughs. This will help determine the suitable location for opening the African restaurent.

#### Methodology

This section of the report is above the report from step 1 to step 6. Every step and sub-steps have been explained in details. Please check the steps for details and clarification. A summary of the steps can be found on the table of content.

#### Results

The results of this work are presented in step 6. In this step, the Foursquare API was used to evaluate and explore the neighborhoods within Manhattan, Brooklyn, and Queens. The results show that:
In Manhattan, there are 3 African resturants all in the Central Harlem neighborhood
In Brooklyn, there is only 1 African restaurant in Ocean Hill neigborhood
In Queens, there is no African restaurent on record

#### Discussions

The three boroughs were chosen because Manhattan is the biggest commercial borough. Queens has the largest land mass (108.1 sq miles) followed by Brooklyn (69.5 sq miles). The two busiest airports in NYC are located in Queens.

Furthermore, according to the demographics of New York City (Wikipedia, 2019), 2 out of every 5 residents in NYC live in Brooklyn and 1 out of every 5 residents live in Queens. Thus, the target borough audience for opening a new African resturant is in Queens and Brooklyn. Manhattan is considered because of its commerical presence.

Given the result, a recommendation for establishing an African restaurent in Queens NYC is made. This rules out the competition challenge and repition of restaurant types and cuisines.

#### Conclusion 

In this study, NYC data was evaluated to determine the most suitable borough for opening an African restaurant. Several python libraries and methods were used to read and evaluate the data. Foursquare API credentials were also used to investigate the neighborhoods in the selected boroughs. The reasons for selecting three boroughs have been provided in the discussion. This work can be extended to the remaining two boroughs. At the end of the study, Queens borough appears to be the most suitable borough for locating the proposed African restaurent. 

#### Delivery Method

The delivery method chosen to show case this work is power point presentation. Not more than 10 slides for a 10 minute presentation. The power point will be divided into:

Introduction/Buisness Problem, Methodology, Results/Discussions, Conclusion/recommendations

It developed and provided upon request.